# INSTALACION DE PAQUETES

In [ ]:
!pip3 install  pandas numpy matplotlib seaborn scipy imblearn requests tqdm scikit-learn umap-learn xgboost lightgbm

# CARGA DE LIBRERIAS

In [ ]:
import database
import eda
import xgbm
import lgbm


# DESCARGA DE LOS DATOS

In [ ]:
database.initialize_database()
cats = database.download_data()

# CARGA DATOS

In [ ]:
database.upload_to_db(cats)

# DESCARGA DE DATOS A DATAFRAME

In [ ]:
df = database.get_dataframe()

# EDA

In [ ]:
df

Transformamos el dataframe

In [ ]:
df = eda.transform_dataframe(df)

In [ ]:
eda.data_resume_info(df=df)

In [ ]:
eda.plot_class_balance(df=df)

In [ ]:
df = eda.drop_low_ink_drawings(df, quantile=0.01)

In [ ]:
eda.plot_class_balance(df=df)

Encontramos que nuestro dataset se encuentra bastante balanceado, donde se garantiza que nuestras dos categorias de entrenamiento cuentan con la misma dimensionalidad. En caso de que haya una categoria con menor cantidad toca buscar metodologias de balanceo como SMOTE y undersampling, o en su defecto aplicar un poco de data augmentation para mejorar.

In [ ]:
eda.plot_average_images(df=df)

sobre pone todas las imagenes que tengamos de la categoria e identifica de manera visual que tanto se parecen. Para el caso del dragon tenemos una sola nube gris, lo que implica que es una imagen compleja donde la interpretabilidad de individuo hace que la imagen cuenta con una alta varianza espacial es decir tenemos que generar un centrado por centro de masa
Para los otros casos se observa que siguen formas similares y tenemos una gran concentración de tinta en las partes caracteristicas de la imagen

In [ ]:
eda.analyze_complexity(df=df)

observando la imagen podemos decir que definitivamente dragon tiene una mayor varianza, es decir una cantidad mayor de diferentes dibujos. Adicional encontramos que tenemos pixeles outlier, quiere decir, gente que dibujo usando mucha titna o poca. Para el caso de dragon observamos que se usa mucha más cantidad de tinta lo que quiere decir más cocnentración en los detalles del dibujo. Manzana tiene una cantidad superior de outliers por debajo del bigote inferior lo que quiere decir que hay más dibujos incompletos o lineas sueltas en el dibujo. Las imagenes tiene una diferencia significativa en sus medias, pero tenemos que se solapan, eso quiere decir, que si se tiene una imagen con una cantidad de tinta similar a la parte en la que se encuentran los boxplot al modelo le sería dificil distinguir los dos grupos.

In [ ]:
eda.test_complexity_significance(df=df)

Con ANOVA confirmamos un Pvalor <0.05 lo que nos indica que rechazamos la hipotesis nula y decimos que por lo menos una de las media de concentración de tinta de nuestros grupos es diferente, lo que nos indica que no tenemos ningun dibujo que tenga en promedio la misma concentración de tinta que los otros grupos

Centramos las imagenes a partir de su centro de masa

In [ ]:
df_centered = eda.preprocess_center_images(df=df)
eda.plot_average_images(df=df_centered)

In [ ]:
eda.analyze_complexity(df=df_centered)

In [ ]:
eda.plot_pixel_variance_heatmap(df=df_centered)

In [ ]:
df_clean = eda.remove_low_variance_pixels(df, threshold=0)

In [ ]:
df_clean.shape

In [ ]:
eda.visualize_umap_projection(df_clean, n_samples=500)

Como sabemos que la base de datos tiene algunos dibujos que están prácticamente vacíos, vamos a proceder a intentar eliminar los que tienen poca tinta, mostrando un heatmap de los dibujos

In [ ]:
eda.data_resume_info(df=df_clean)

# xgboost

In [ ]:
X_train, X_test, y_train, y_test = xgbm.split_df(df=df_clean)

In [ ]:
xgbm.train_model(X_train, y_train, X_test, y_test)

In [ ]:
xgbm.run_grid_search(X_train, y_train, X_test, y_test)